In [282]:
import numpy as np
import pandas as pd

In [283]:
def id(number):
    if len(str(number))==1:
        return '00'+str(number)
    else:
        return '0'+str(number)

In [284]:
foods = []
dexcoms = []
to_return = {}
for i in range(1, 17):
    current_id = id(i)
    foods.append(pd.read_csv(f'data/{current_id}/Food_Log_{current_id}.csv'))
    dexcoms.append(pd.read_csv(f'data/{current_id}/Dexcom_{current_id}.csv'))

In [285]:
for i in range(16):
    dexcom = dexcoms[i]
    food = foods[i]
    dexcom['time'] = pd.to_datetime(dexcom['Timestamp (YYYY-MM-DDThh:mm:ss)'])
    dexcom = dexcom[dexcom['time'].notna()]
    food['time'] = pd.to_datetime(food['time_begin'])
    food = food[food['time'].notna()]

    #food = food.groupby('time', as_index=False)[['calorie', 'total_carb', 'dietary_fiber', 'sugar', 'protein', 'total_fat']].sum()
    food = food.groupby('time', as_index=False).agg({
        'calorie': 'sum',
        'total_carb': 'sum',
        'dietary_fiber': 'sum',
        'sugar': 'sum',
        'protein': 'sum',
        'total_fat': 'sum',
        'logged_food': list,     # example column to aggregate as list
    })
    dexcom = dexcom.sort_values('time')
    food = food.sort_values('time')

    # Create a results dataframe to store our findings
    results = []
    for _, food_elem in food.iterrows():
        # Find the closest timestamp in df that comes before food_time
        food_time = food_elem['time']
        mask = dexcom['time'] <= food_time
        if not mask.any():
            continue  # Skip if no earlier timestamp exists
        closest_time = dexcom.loc[mask, 'time'].max()
        basis_glucose = dexcom.loc[dexcom['time'] == closest_time, 'Glucose Value (mg/dL)'].iloc[0]
    
        # Find all records within one hour after the closest_time
        one_hour_later = closest_time + pd.Timedelta(hours=1)
        time_window = dexcom[(dexcom['time'] >= closest_time) & (dexcom['time'] <= one_hour_later)]
    
        # Calculate sugar changes relative to the basis
        for _, row in time_window.iterrows():
            results.append({
                'participant_id': id(i+1),
                'food_time': food_time,
                'basis_time': closest_time,
                'current_time': row['time'],
                'basis_glucose': basis_glucose,
                'current_glucose': row['Glucose Value (mg/dL)'],
                'sugar_change': row['Glucose Value (mg/dL)'] - basis_glucose,
                'minutes_from_basis': round(int((row['time'] - closest_time).total_seconds() / 60)/5)*5,
                'calorie': food_elem['calorie'],
                'total_carb': food_elem['total_carb'],
                #'dietary_fiber': food_elem['dietary_fiber'],
                'sugar': food_elem['sugar'],
                'protein': food_elem['protein'],
                #'total_fat': food_elem['total_fat'],
                'logged_food': food_elem['logged_food']
            })
    results_df = pd.DataFrame(results)
    to_return[id(i+1)] = results_df

In [286]:
for i in range(1, 17):
    if i==3:
        continue
    to_return[id(i)].to_csv(f'data/{id(i)}/summary{id(i)}.csv')

In [287]:
male_high = []
for i in [9, 11]:
    male_high.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
male_high_df = pd.concat(male_high).drop('Unnamed: 0', axis=1)
male_high_df = male_high_df[(pd.to_datetime(male_high_df['food_time'])-pd.to_datetime(male_high_df['basis_time']))< pd.Timedelta(minutes=5)]
male_high_df = pd.pivot_table(male_high_df, values=['sugar_change'],
                              columns='minutes_from_basis',
                              index=['participant_id', 'basis_time', 'calorie', 'total_carb', 'sugar', 'protein'])
male_high_df.columns = [f'{col[1]}' for col in male_high_df.columns]
male_high_df = male_high_df.reset_index()
male_high_df.to_csv(f'data/male_high.csv')

In [288]:
male_low = []
for i in [2, 12, 14]:
    male_low.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
male_low_df = pd.concat(male_low).drop('Unnamed: 0', axis=1)
male_low_df = male_low_df[(pd.to_datetime(male_low_df['food_time'])-pd.to_datetime(male_low_df['basis_time']))< pd.Timedelta(minutes=5)]
male_low_df = pd.pivot_table(male_low_df, values=['sugar_change'],
                              columns='minutes_from_basis',
                              index=['participant_id', 'basis_time', 'calorie', 'total_carb', 'sugar', 'protein'])
male_low_df.columns = [f'{col[1]}' for col in male_low_df.columns]
male_low_df = male_low_df.reset_index()
male_low_df.to_csv(f'data/male_low.csv')

In [289]:
female_high = []
for i in [4, 5, 6, 10]:
    female_high.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
female_high_df = pd.concat(female_high).drop('Unnamed: 0', axis=1)
female_high_df = female_high_df[(pd.to_datetime(female_high_df['food_time'])-pd.to_datetime(female_high_df['basis_time']))< pd.Timedelta(minutes=5)]
female_high_df = pd.pivot_table(female_high_df, values=['sugar_change'],
                              columns='minutes_from_basis',
                              index=['participant_id', 'basis_time', 'calorie', 'total_carb', 'sugar', 'protein'])
female_high_df.columns = [f'{col[1]}' for col in female_high_df.columns]
female_high_df = female_high_df.reset_index()
female_high_df.to_csv(f'data/female_high.csv')

In [290]:
female_low = []
for i in [1, 8]:
    female_low.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
female_low_df = pd.concat(female_low).drop('Unnamed: 0', axis=1)
female_low_df = female_low_df[(pd.to_datetime(female_low_df['food_time'])-pd.to_datetime(female_low_df['basis_time']))< pd.Timedelta(minutes=5)]
female_low_df = pd.pivot_table(female_low_df, values=['sugar_change'],
                              columns='minutes_from_basis',
                              index=['participant_id', 'basis_time', 'calorie', 'total_carb', 'sugar', 'protein'])
female_low_df.columns = [f'{col[1]}' for col in female_low_df.columns]
female_low_df = female_low_df.reset_index()
female_low_df.to_csv(f'data/female_low.csv')

In [ ]:
def group_participants(participants_id, csv_name):
    dfs = []
    for i in participants_id:
        dfs.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
    all_df = pd.concat(dfs).drop('Unnamed: 0', axis=1)
    all_df = all_df[(pd.to_datetime(all_df['food_time'])-pd.to_datetime(all_df['basis_time']))< pd.Timedelta(minutes=5)]
    all_df = pd.pivot_table(all_df, values=['sugar_change'],
                                columns='minutes_from_basis',
                                index=['participant_id', 'basis_time', 'calorie', 'total_carb', 'sugar', 'protein'])
    all_df.columns = [f'{col[1]}' for col in all_df.columns]
    all_df = all_df.reset_index()
    #all_df.to_csv(csv_name)
    return all_df

In [ ]:
df = group_participants([4, 5, 6, 10], 'data/female_high.csv')
df['calorie']

,participant_id,basis_time,calorie,total_carb,sugar,protein,0,5,10,15,20,25,30,35,40,45,50,55,60
0,4,2020-02-27 13:26:20,34.0,8.9,7.6,0.4,0.0,0.0,2.0,4.0,6.0,6.0,5.0,4.0,4.0,2.0,0.0,-2.0,NaN
1,4,2020-02-27 15:56:20,852.0,0.0,0.0,102.0,0.0,0.0,0.0,-2.0,-3.0,-2.0,-2.0,-3.0,-6.0,-8.0,-9.0,-13.0,NaN
2,4,2020-02-27 17:56:21,68.0,17.0,15.0,1.6,0.0,1.0,-1.0,-6.0,-14.0,-20.0,-18.0,-9.0,2.0,12.0,20.0,30.0,35.0
3,4,2020-02-28 07:56:19,280.0,56.5,24.0,8.0,0.0,0.0,1.0,2.0,3.0,4.0,4.0,4.0,4.0,3.0,2.0,0.0,-1.0
4,4,2020-02-28 12:41:18,1613.0,134.8,30.1,125.8,0.0,0.0,-1.0,-3.0,-6.0,-7.0,-9.0,-10.0,-11.0,-11.0,-8.0,-6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,10,2020-03-29 08:29:03,0.0,0.4,0.0,0.0,0.0,14.0,22.0,26.0,22.0,13.0,-1.0,-15.0,-19.0,-14.0,-4.0,2.0,4.0
183,10,2020-03-29 12:49:05,831.0,92.0,6.8,48.0,0.0,-3.0,-4.0,-1.0,3.0,7.0,14.0,21.0,26.0,29.0,35.0,48.0,68.0
184,10,2020-03-29 15:34:04,2.4,0.0,0.0,0.3,0.0,-10.0,-14.0,-15.0,-13.0,-8.0,-1.0,4.0,7.0,6.0,2.0,-3.0,-8.0
185,10,2020-03-29 17:29:04,300.0,46.0,26.0,5.4,0.0,-11.0,-22.0,-31.0,-36.0,-39.0,-37.0,-29.0,-16.0,-3.0,7.0,13.0,16.0
